In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

In [2]:
sc=SparkContext()

23/07/24 17:09:21 WARN Utils: Your hostname, shua-VivoBook-ASUSLaptop-X513EAN-K513EA resolves to a loopback address: 127.0.1.1; using 10.144.125.210 instead (on interface wlo1)
23/07/24 17:09:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/24 17:09:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
rdd = sc.parallelize([('b', 1), ('c', 2) , ('a', 3)])
print(rdd.sortBy(lambda el: el[0]).collect()) # Sort by key
print(rdd.sortBy(lambda el: el[1]).collect()) # Sort by value

[('a', 3), ('b', 1), ('c', 2)]
[('b', 1), ('c', 2), ('a', 3)]


In [3]:
def adsMap(el):
    els =el.split(',')
    return (els[1], els[0])
def channelsMap(el):
    els =el.split(',')
    return (els[0], els[1])
def printRes(rdd):
    for el in rdd.collect():
        print((el[0], el[1]))

In [4]:
ads = sc.textFile("data/ads.txt").map(adsMap)
channels = sc.textFile("data/channels.txt").map(channelsMap)

In [9]:
ads.take(5)

[('HVT', 'Air Taser Model 34000'),
 ('TLK', 'Air Taser Model 34000'),
 ('IJU', 'Air Taser Model 34000'),
 ('KHR', 'Air Taser Model 34000'),
 ('LEO', 'Air Taser Model 34000')]

In [8]:
channels.take(5)

[('GID', '14312'),
 ('PAY', '71628'),
 ('BRG', '71971'),
 ('YCR', '96656'),
 ('FRV', '44468')]

In [5]:
input = ads.join(channels)
input.take(10)

[('TLK', ('Air Taser Model 34000', '59811')),
 ('TLK', ('Apple iPad', '59811')),
 ('TLK', ('Bellsouth IBM Simon', '59811')),
 ('TLK', ('Digital Thermometer', '59811')),
 ('TLK', ('iRobot Roomba', '59811')),
 ('TLK', ('Magellan NAV 1000', '59811')),
 ('TLK', ('Motorola PageWriter 2000', '59811')),
 ('TLK', ('Sony Mavica MVC-FD5', '59811')),
 ('TLK', ('Spot Personal Tracker', '59811')),
 ('TLK', ('Velcro', '59811'))]

In [10]:
second = input.map(lambda el: (el[1][0], int(el[1][1])))
second.take(5)

[('Air Taser Model 34000', 59811),
 ('Apple iPad', 59811),
 ('Bellsouth IBM Simon', 59811),
 ('Digital Thermometer', 59811),
 ('iRobot Roomba', 59811)]

In [12]:
res = second.reduceByKey(lambda sum, el: sum+el)
res.take(10)

[('Apple iPad', 345245),
 ('iRobot Roomba', 839835),
 ('Sony Mavica MVC-FD5', 2250393),
 ('Velcro', 1841346),
 ('Clock Radio', 1254039),
 ('GRiD Compass 1101', 665914),
 ('Dirt Devil Hand Vac', 550769),
 ('Apple AirPort', 1391962),
 ('Epson HX-20', 2153635),
 ('Radio Shack TRS-80 Model 100', 1581662)]

In [13]:
res = second.groupByKey().mapValues(lambda vals: sum(vals))
res.take(10)

[('Apple iPad', 345245),
 ('iRobot Roomba', 839835),
 ('Sony Mavica MVC-FD5', 2250393),
 ('Velcro', 1841346),
 ('Clock Radio', 1254039),
 ('GRiD Compass 1101', 665914),
 ('Dirt Devil Hand Vac', 550769),
 ('Apple AirPort', 1391962),
 ('Epson HX-20', 2153635),
 ('Radio Shack TRS-80 Model 100', 1581662)]

In [73]:
rdd = sc.parallelize([('A', 10), ('B', 20), ('C', 30)], 3)

In [74]:
rdd.collect()

[('A', 10), ('B', 20), ('C', 30)]

In [75]:
rdd.map(lambda x: x[1]).collect()

[10, 20, 30]

In [50]:
rdd.flatMap(lambda el: el).collect()

['A', 10, 'B', 20, 'C', 30, 'D', 40]

In [77]:
grouped = rdd.groupByKey()
for el in grouped.collect():
    print(el[0], list(el[1]))

B [20]
C [30]
A [10]


In [79]:
rdd = sc.parallelize([('b', 10), ('a',13), ('c',12), ('a', 20)],2)
mapped = rdd.map(lambda el: (int(el[1]/10), el[0]))
mapped.collect()

[(1, 'b'), (1, 'a'), (1, 'c'), (2, 'a')]

In [81]:
grouped = mapped.reduceByKey(lambda newel, el: min(newel, el))
grouped.collect()

[(2, 'a'), (1, 'a')]